In [1]:
import datetime
import json
import pandas as pd
import requests

from mcmetadata import extract
import mediacloud.api

from tqdm.notebook import trange  # to display a progress bar
import ipywidgets as widgets
from IPython.display import display

# Init the connection
mc = mediacloud.api.DirectoryApi('56196a395ee77c33a296073fa08e72f541362a10')

CSV_FILE = 'Query_NYT_13y_surveillance.csv'
JSON_FILE = 'data.json'

# Read the query.csv file to obtain information of each item to dump
df_query = pd.read_csv(CSV_FILE)

AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [ ]:
size_query = df_query.shape[0]
print(size_query, "URLs in the CSV file :", CSV_FILE)

In [ ]:
import time
import random

# Create the result dataframe
df_result = pd.DataFrame(columns=["year", "title", "url", "text_content"])

# Create the empty JSON file
with open(JSON_FILE, "w") as f:
    f.write("[")

with open("log.txt", "w") as f:
    f.write("")

# Création du widget d'erreurs
errors_widget = widgets.IntText(description="Erreurs sur URL:", value=0)
display(errors_widget)

rec = 0

for counter in trange(0, size_query):
    # Query the MediaCloud API with each item
    url_data = df_query.iloc[counter].url

    url = "https://archive.org/wayback/available"
    params = {
        "url": url_data,
        "timeout": "30",
        "closest": "either",
        "status_code": "200",
    }
    headers = {"accept": "application/json"}

    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    closest_snapshot = data.get("archived_snapshots", {}).get("closest")
    if closest_snapshot:
        closest_url = closest_snapshot.get("url")
        is_available = closest_snapshot.get("available")
        if is_available == True:
            try:
                metadata = extract(url=closest_url)
                metadata["publication_date"] = metadata["publication_date"].year
                metadata["url"] = url_data
                metadata["original_url"] = url_data

                # Append data to the result dataframe
                list_metadata = []
                for field in [
                    "publication_date",
                    "article_title",
                    "url",
                    "text_content",
                ]:
                    list_metadata.append(metadata[field])
                # Append data to the result dataframe
                df_result.loc[counter] = list_metadata
                # And write data in JSON file
                with open(JSON_FILE, "a") as f:
                    if rec == 1:
                        f.write(",")
                    else:
                        rec = 1
                    json.dump(metadata, f)
                    f.write("\n")
            except Exception as e:
                # print(f'Erreur -> "{e}", pour l\'url : {url_data}' )
                errors_widget.value += 1
                log_message = f"{datetime.datetime.now()} - {e} ; {url_data}\n"
                with open("log.txt", "a") as log_file:
                    log_file.write(log_message)
    time.sleep(random.uniform(5.0, 10.0))

with open(JSON_FILE, "a") as f:
    f.write("]")

with open("log.txt", "a") as log_file:
    log_file.write("----------------------------------\n")
print(df_result.describe)

In [ ]:
# Export dataframe to a feather format file
df_result.reset_index().to_feather('results.feather')

print("\n-----------------------------------")
print("  Process complete !")